In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\Hima Bindu N\Downloads\archive\AmesHousing.csv")

df.columns = df.columns.str.replace(" ", "")

df.isnull().sum()

num_cols = df.select_dtypes(include=np.number).columns
cat_cols = df.select_dtypes(include='object').columns

df[num_cols] = df[num_cols].fillna(df[num_cols].median())
df[cat_cols] = df[cat_cols].fillna(df[cat_cols].mode().iloc[0])

df.isnull().sum()

Order            0
PID              0
MSSubClass       0
MSZoning         0
LotFrontage      0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 82, dtype: int64